In [1]:
import torch
import io
import os
import json
from botorch.models import SingleTaskGP, FixedNoiseGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.multi_objective import ExpectedHypervolumeImprovement
from gpytorch.mlls import ExactMarginalLogLikelihood
import asyncio
dtype = torch.float64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def x_to_filename(x):
    return f'{x[0]}_{x[1]}_{x[2]}_{x[3]}.json'

def filename_to_x(filename):
    return [float(x) for x in filename[:-5].split('_')]

async def evaluate_profile(x):
    filename=x_to_filename(x)
    #display(filename)
    existing_result_files=os.listdir("case/results")
    existing_results_x=[filename_to_x(filename) for filename in existing_result_files]
    distances=[torch.dist(torch.tensor(x),torch.tensor(existing_x)).item() for existing_x in existing_results_x]
    #display(distances)
    if min(distances)>0.001:
        config=json.dumps({
            'p':x[0],
            'm':x[1],
            't':x[2],
            'a':x[3]
        })
        p=await asyncio.create_subprocess_shell(f'cd case; ./evaluate_airfoil.sh \'{config}\'',stdout=asyncio.subprocess.PIPE)
        await p.wait()
        #os.system(f'cd case; ./evaluate_airfoil.sh \'{config}\'')
    min_distance_index=distances.index(min(distances))
    with open("case/results/"+existing_result_files[min_distance_index]) as file:
        result=json.load(file)
        #display(result)
        return [result['C_L'],result['C_L']/result['C_D']]


await evaluate_profile([0.42,0.05,0.18,14])

[1.44476, 10.586571506034248]

In [2]:
x_lims=[(0.1,0.9),(-0.2,0.2),(0.05,0.3),(0.01,35)]

# initial_X = torch.rand(6,4)
# for d in range(4):
#     initial_X[:,d]=x_lims[d][0]+initial_X[:,d]*(x_lims[d][1]-x_lims[d][0])

# get initial X from filenames in case/results folder
initial_X=torch.tensor([filename_to_x(filename) for filename in os.listdir("case/results")],dtype=dtype,device=device)
# take six random points from the initial X
initial_X = initial_X[torch.randperm(initial_X.shape[0])[:20]]

display(initial_X)
y=await asyncio.gather(*[evaluate_profile(x.tolist()) for x in initial_X])

tensor([[ 4.2000e-01,  1.0000e-01,  1.8000e-01,  1.2000e+01],
        [ 1.6297e-01, -1.9246e-01,  6.6998e-02,  2.2169e+00],
        [ 4.2000e-01,  1.0000e-01,  1.8000e-01,  1.5000e+01],
        [ 4.2000e-01,  1.0000e-01,  1.8000e-01,  2.0000e+01],
        [ 4.0000e-01,  5.0000e-02,  1.5000e-01,  5.0000e+01],
        [ 5.2345e-01,  4.2727e-02,  1.9550e-01,  1.5996e+01],
        [ 4.0000e-01,  5.0000e-02,  1.5000e-01,  6.0000e+01],
        [ 4.0000e-01,  5.0000e-02,  1.5000e-01,  4.0000e+01],
        [ 3.3545e-01,  3.0853e-02,  2.7415e-01,  2.4670e+01],
        [ 2.4970e-01, -5.1559e-02,  1.4601e-01,  3.0229e+00],
        [ 4.1784e-01, -6.5921e-02,  2.1528e-01,  2.2247e+01],
        [ 4.0000e-01,  5.0000e-02,  1.5000e-01,  2.0000e+01],
        [ 4.2000e-01,  1.0000e-01,  1.8000e-01,  1.0000e-02],
        [ 4.2000e-01,  1.0000e-01,  3.0000e-01,  3.0000e+01],
        [ 4.2000e-01,  5.0000e-02,  1.8000e-01,  1.1000e+01],
        [ 1.9553e-01,  1.3699e-01,  7.0834e-02,  6.9063e+00],
        

In [3]:
bounds = torch.stack([torch.tensor(x_lims)[:,0], torch.tensor(x_lims)[:,1]])
bounds

tensor([[ 1.0000e-01, -2.0000e-01,  5.0000e-02,  1.0000e-02],
        [ 9.0000e-01,  2.0000e-01,  3.0000e-01,  3.5000e+01]])

In [6]:

from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
train_X=initial_X
train_Y = torch.tensor(y,dtype=dtype,device=device)
#print(train_X)
#print(train_Y)
N_BATCH=3
for iteration in range(1, N_BATCH + 1):
    #gp = SingleTaskGP(train_X, train_Y)
    train_Yvar = torch.full_like(train_Y, 0.01)
    gp= FixedNoiseGP(train_X, train_Y, train_Yvar)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    #test_X=torch.rand(1,4)
    #print(test_X,": ",gp.posterior(train_X).mean,"+-",gp.posterior(train_X).variance.sqrt())

    #

    #acquisition_function = UpperConfidenceBound(gp, beta=0.1)
    partitioning=FastNondominatedPartitioning(ref_point=torch.tensor([0.0, 0.0]),Y=train_Y)
    acquisition_function = ExpectedHypervolumeImprovement(
        gp, ref_point=torch.tensor([0.0, 0.0]),
        partitioning=partitioning)

    candidate, acq_value = optimize_acqf(
        acquisition_function, bounds=bounds, q=1, num_restarts=1, batch_initial_conditions=train_X
    )
    print(candidate,": ",acq_value)
    predicted_Y=gp.posterior(candidate.unsqueeze(0)).mean
    prediceted_Y_var=gp.posterior(candidate.unsqueeze(0)).variance
    print(predicted_Y,"+-",prediceted_Y_var.sqrt())
    actual_value=await evaluate_profile(candidate.tolist())
    print(actual_value)
    # add new data to training data
    train_X = torch.cat([train_X, candidate.unsqueeze(0)])
    train_Y = torch.cat([train_Y, torch.tensor(actual_value,dtype=dtype,device=device).unsqueeze(0)])
    print(train_Y)

/home/olep/.local/lib/python3.10/site-packages/botorch/acquisition/multi_objective/analytic.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ref_point = torch.tensor(


tensor([ 0.4290,  0.1162,  0.2965, 19.9220], dtype=torch.float64) :  tensor(3.5622, dtype=torch.float64)
tensor([[ 2.0016, 22.7987]], dtype=torch.float64, grad_fn=<TransposeBackward0>) +- tensor([[0.3329, 3.0569]], dtype=torch.float64, grad_fn=<SqrtBackward0>)
[1.90987, 14.748828121983427]
tensor([[ 1.6956, 12.4764],
        [-0.5486, -1.7773],
        [ 1.8373, 17.3114],
        [ 2.0099, 25.4021],
        [ 1.2502,  1.7287],
        [ 1.3467,  8.2575],
        [ 0.8679,  1.0383],
        [ 1.5748,  3.0283],
        [ 1.4417,  6.1830],
        [-0.1452, -1.0555],
        [ 0.4364,  0.7918],
        [ 1.6922, 12.6588],
        [ 0.8315,  6.2962],
        [ 1.8565, 11.3555],
        [ 1.2629,  8.9512],
        [ 1.4649,  8.3809],
        [ 2.0008,  9.3529],
        [ 1.8688, 10.5315],
        [ 0.9425,  5.2468],
        [ 1.5422, 12.6082],
        [ 1.9099, 14.7488]], dtype=torch.float64)


/home/olep/.local/lib/python3.10/site-packages/botorch/acquisition/multi_objective/analytic.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ref_point = torch.tensor(


tensor([ 0.3121,  0.1226,  0.1505, 19.9295], dtype=torch.float64) :  tensor(2.4400, dtype=torch.float64)
tensor([[ 1.9633, 21.3649]], dtype=torch.float64, grad_fn=<TransposeBackward0>) +- tensor([[0.2936, 3.5038]], dtype=torch.float64, grad_fn=<SqrtBackward0>)


[1.69216, 12.658761922573408]
tensor([[ 1.6956, 12.4764],
        [-0.5486, -1.7773],
        [ 1.8373, 17.3114],
        [ 2.0099, 25.4021],
        [ 1.2502,  1.7287],
        [ 1.3467,  8.2575],
        [ 0.8679,  1.0383],
        [ 1.5748,  3.0283],
        [ 1.4417,  6.1830],
        [-0.1452, -1.0555],
        [ 0.4364,  0.7918],
        [ 1.6922, 12.6588],
        [ 0.8315,  6.2962],
        [ 1.8565, 11.3555],
        [ 1.2629,  8.9512],
        [ 1.4649,  8.3809],
        [ 2.0008,  9.3529],
        [ 1.8688, 10.5315],
        [ 0.9425,  5.2468],
        [ 1.5422, 12.6082],
        [ 1.9099, 14.7488],
        [ 1.6922, 12.6588]], dtype=torch.float64)


/home/olep/.local/lib/python3.10/site-packages/botorch/acquisition/multi_objective/analytic.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ref_point = torch.tensor(


tensor([ 0.5542,  0.1062,  0.1628, 20.0149], dtype=torch.float64) :  tensor(3.3208, dtype=torch.float64)
tensor([[ 1.9985, 23.0282]], dtype=torch.float64, grad_fn=<TransposeBackward0>) +- tensor([[0.3060, 2.9089]], dtype=torch.float64, grad_fn=<SqrtBackward0>)


[2.00995, 25.40205117187253]
tensor([[ 1.6956, 12.4764],
        [-0.5486, -1.7773],
        [ 1.8373, 17.3114],
        [ 2.0099, 25.4021],
        [ 1.2502,  1.7287],
        [ 1.3467,  8.2575],
        [ 0.8679,  1.0383],
        [ 1.5748,  3.0283],
        [ 1.4417,  6.1830],
        [-0.1452, -1.0555],
        [ 0.4364,  0.7918],
        [ 1.6922, 12.6588],
        [ 0.8315,  6.2962],
        [ 1.8565, 11.3555],
        [ 1.2629,  8.9512],
        [ 1.4649,  8.3809],
        [ 2.0008,  9.3529],
        [ 1.8688, 10.5315],
        [ 0.9425,  5.2468],
        [ 1.5422, 12.6082],
        [ 1.9099, 14.7488],
        [ 1.6922, 12.6588],
        [ 2.0099, 25.4021]], dtype=torch.float64)
